# Stretch Workshop - ALL STAR LEVEL

For those with prior experience with SQL, or those who want to increase their SQL proficiency, this workshop begins by introducing SQL Window Functions and includes a few questions at the end -- for you to try out. Good luck!

# Window Function

Window functions perform calculations on a set of rows that are somehow related to the current row. They are similar to aggregate functions, but they don't group rows into one row -- the rows retain their identities.

We'll continue working with our General Store database in this lecture.

In [1]:
import pandas as pd
import sqlite3

df = pd.read_csv("../data/sales2019.csv")
conn = sqlite3.connect('../data/generalstore.sqlite',)
c = conn.cursor()
df.to_sql('sales2019',conn,if_exists='replace')

In [2]:
# let's return the sum of total sales  together with individual sales

pd.read_sql("""
            select customerID, 
            item_description,
            Date,
            Sales2019,
            SUM(Sales2019) OVER () TotSales
            from sales2019
            """, conn)

,CUSTOMERID,item_description,Date,Sales2019,TotSales
0,1003,Bed bug spray,11/15/2019,5,1200
1,1003,Hammer,11/16/2019,10,1200
2,1003,Giant roach spray,12/15/2019,5,1200
3,1004,Chocolate,12/20/2019,50,1200
4,1004,Chocolate,12/30/2019,50,1200
5,1005,Potions,11/15/2019,20,1200
6,1008,Books,11/17/2019,15,1200
7,1011,Bow,12/5/2019,700,1200
8,1011,Arrows,12/10/2019,150,1200
9,1011,Tote bag,12/30/2019,20,1200


The keyword OVER() indicates that the SUM() should be used as a **window function**.

# ROW_NUMBER

In [3]:
# assign row number to each record

pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   ROW_NUMBER() OVER () as row_num
                   FROM Sales2019
                   ORDER BY row_num""", conn)

,CUSTOMERID,item_description,Date,row_num
0,1003,Bed bug spray,11/15/2019,1
1,1003,Hammer,11/16/2019,2
2,1003,Giant roach spray,12/15/2019,3
3,1004,Chocolate,12/20/2019,4
4,1004,Chocolate,12/30/2019,5
5,1005,Potions,11/15/2019,6
6,1008,Books,11/17/2019,7
7,1011,Bow,12/5/2019,8
8,1011,Arrows,12/10/2019,9
9,1011,Tote bag,12/30/2019,10



Using the query above, select only "chocolate" items with even rows numbers.

In [4]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   ROW_NUMBER() OVER () as row_num
                   FROM Sales2019
                   WHERE item_description LIKE '%chocolate%' AND rowid>0
                   ORDER BY row_num""", conn)

,CUSTOMERID,item_description,Date,row_num
0,1004,Chocolate,12/20/2019,1
1,1004,Chocolate,12/30/2019,2


In [5]:
# more often, we'd want to assign row number to each group of records
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   ROW_NUMBER() OVER ( 
                       PARTITION BY CUSTOMERID
                       ORDER BY item_description, Date
                   ) as row_num
                   FROM Sales2019""", conn)

# partition divides records by customerID
# ORDER BY sorts items in each partition by description & DAte
# ROW_NUMBER() assigns each row in each partition an integer

,CUSTOMERID,item_description,Date,row_num
0,1003,Bed bug spray,11/15/2019,1
1,1003,Giant roach spray,12/15/2019,2
2,1003,Hammer,11/16/2019,3
3,1004,Chocolate,12/20/2019,1
4,1004,Chocolate,12/30/2019,2
5,1005,Potions,11/15/2019,1
6,1008,Books,11/17/2019,1
7,1011,Arrows,12/10/2019,1
8,1011,Bow,12/5/2019,2
9,1011,Tote bag,12/30/2019,3


# DENSE_RANK

In [6]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019,
                   DENSE_RANK() OVER (  
                       ORDER BY CUSTOMERID
                   ) as row_num
                   FROM Sales2019""", conn) 
# can also use with PARTITION BY

,CUSTOMERID,item_description,Date,Sales2019,row_num
0,1003,Bed bug spray,11/15/2019,5,1
1,1003,Hammer,11/16/2019,10,1
2,1003,Giant roach spray,12/15/2019,5,1
3,1004,Chocolate,12/20/2019,50,2
4,1004,Chocolate,12/30/2019,50,2
5,1005,Potions,11/15/2019,20,3
6,1008,Books,11/17/2019,15,4
7,1011,Bow,12/5/2019,700,5
8,1011,Arrows,12/10/2019,150,5
9,1011,Tote bag,12/30/2019,20,5


# RANK

In [7]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019,
                   RANK() OVER (  
                       ORDER BY CUSTOMERID
                   ) as row_num
                   FROM Sales2019""", conn)

,CUSTOMERID,item_description,Date,Sales2019,row_num
0,1003,Bed bug spray,11/15/2019,5,1
1,1003,Hammer,11/16/2019,10,1
2,1003,Giant roach spray,12/15/2019,5,1
3,1004,Chocolate,12/20/2019,50,4
4,1004,Chocolate,12/30/2019,50,4
5,1005,Potions,11/15/2019,20,6
6,1008,Books,11/17/2019,15,7
7,1011,Bow,12/5/2019,700,8
8,1011,Arrows,12/10/2019,150,8
9,1011,Tote bag,12/30/2019,20,8


## RANK vs DENSE_RANK vs ROW_NUMBER

These functions are similar because they are used to retrieve an increasing integer value. They all require ORDER BY to return proper results. There are however slight differences in their behavior in an event of a tie.

RANK: When there is a tie between N previous rows, RANK skips the next N-1 positions before increasing the counter. For example, we have a tie for CUSTOMERID 1003, all records for this customer have 1 as their value, and 4 for CUSTOMERID 1004.

DENSE_RANK: This function doesn't skip any ranks but will assign duplicate values to the same CUSTOMERID appearing mutliple times.

ROW_NUMBER: Will return a new integer value for each row.

How would these functions behave when there are no duplicate entries in your table?

# FIRST_VALUE

In [8]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019,
                   FIRST_VALUE(Sales2019) OVER (  
                       ORDER BY Sales2019
                   ) as smallestSales
                   FROM Sales2019""", conn)

,CUSTOMERID,item_description,Date,Sales2019,smallestSales
0,1003,Bed bug spray,11/15/2019,5,5
1,1003,Giant roach spray,12/15/2019,5,5
2,1018,Bed bug spray,11/20/2019,5,5
3,1003,Hammer,11/16/2019,10,5
4,1008,Books,11/17/2019,15,5
5,1005,Potions,11/15/2019,20,5
6,1011,Tote bag,12/30/2019,20,5
7,1017,Potions,12/5/2019,20,5
8,1004,Chocolate,12/20/2019,50,5
9,1004,Chocolate,12/30/2019,50,5


In [9]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019,
                   FIRST_VALUE(Sales2019) OVER (  
                       PARTITION BY CUSTOMERID
                       ORDER BY Sales2019
                   ) as smallestSales
                   FROM Sales2019""", conn)

,CUSTOMERID,item_description,Date,Sales2019,smallestSales
0,1003,Bed bug spray,11/15/2019,5,5
1,1003,Giant roach spray,12/15/2019,5,5
2,1003,Hammer,11/16/2019,10,5
3,1004,Chocolate,12/20/2019,50,50
4,1004,Chocolate,12/30/2019,50,50
5,1005,Potions,11/15/2019,20,20
6,1008,Books,11/17/2019,15,15
7,1011,Tote bag,12/30/2019,20,20
8,1011,Arrows,12/10/2019,150,20
9,1011,Bow,12/5/2019,700,20


# LEAD/LAG

In [10]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019
                   FROM Sales2019
                   ORDER BY CustomerID, Date""", conn)

,CUSTOMERID,item_description,Date,Sales2019
0,1003,Bed bug spray,11/15/2019,5
1,1003,Hammer,11/16/2019,10
2,1003,Giant roach spray,12/15/2019,5
3,1004,Chocolate,12/20/2019,50
4,1004,Chocolate,12/30/2019,50
5,1005,Potions,11/15/2019,20
6,1008,Books,11/17/2019,15
7,1011,Arrows,12/10/2019,150
8,1011,Tote bag,12/30/2019,20
9,1011,Bow,12/5/2019,700


In [11]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019,
                   LEAD(Sales2019,1,0) OVER (
                       PARTITION BY CustomerID
                       ORDER BY Date
                   ) as NextDateSales
                   FROM Sales2019
                   ORDER BY CustomerID, Date""", conn)


,CUSTOMERID,item_description,Date,Sales2019,NextDateSales
0,1003,Bed bug spray,11/15/2019,5,10
1,1003,Hammer,11/16/2019,10,5
2,1003,Giant roach spray,12/15/2019,5,0
3,1004,Chocolate,12/20/2019,50,50
4,1004,Chocolate,12/30/2019,50,0
5,1005,Potions,11/15/2019,20,0
6,1008,Books,11/17/2019,15,0
7,1011,Arrows,12/10/2019,150,20
8,1011,Tote bag,12/30/2019,20,700
9,1011,Bow,12/5/2019,700,0


In [12]:
pd.read_sql("""
            SELECT CUSTOMERID,
                   item_description,
                   Date,
                   Sales2019,
                   LAG(Sales2019,1,0) OVER (
                       PARTITION BY CustomerID
                       ORDER BY Date
                   ) as NextDateSales
                   FROM Sales2019
                   ORDER BY CustomerID, Date""", conn)

,CUSTOMERID,item_description,Date,Sales2019,NextDateSales
0,1003,Bed bug spray,11/15/2019,5,0
1,1003,Hammer,11/16/2019,10,5
2,1003,Giant roach spray,12/15/2019,5,10
3,1004,Chocolate,12/20/2019,50,0
4,1004,Chocolate,12/30/2019,50,50
5,1005,Potions,11/15/2019,20,0
6,1008,Books,11/17/2019,15,0
7,1011,Arrows,12/10/2019,150,0
8,1011,Tote bag,12/30/2019,20,150
9,1011,Bow,12/5/2019,700,20


# Stretch Goal Questions

1. Find a way to bring all data together in one table
2. Calculate sales in 2021 as percent of total sales in 2021 for each customer
3. Return only customers who spent the second most money in all years from each occupation if there are more than one customer. Also include occupations with only one customer